In [4]:
import os, datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
generated_image_dir = '/data/darknet/python/' + time_str + '_image_fillter_obj/'
generated_xml_dir = '/data/darknet/python/' + time_str + '_xml_fillter_obj/'

time_str = '20190123'
xml_dir = '/data/darknet/python/' + time_str + '_xmls_replaced_ordinary_obj_with_bkg/'
image_dir = '/data/darknet/python/' + time_str + '_images_replaced_ordinary_obj_with_bkg/'

os.system('mkdir -p ' + generated_xml_dir)
os.system('mkdir -p ' + generated_image_dir)

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
GENE_IMG_DIR = generated_image_dir
GENE_XML_DIR = generated_xml_dir
BKG_IMG_DIR = '/data/darknet/python/background/'

# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    return img_obj

def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()
bkg_images = os.listdir(BKG_IMG_DIR)
bkg_images.sort()

classes_ = ['bus', 'car', 'truck', 'motorbike', 'bicycle', 'person']

last_frame_bboxes = []
for xml_name in seed_xml_names:
    #print(xml_name)       
    if(not xml_name.endswith('.xml')):
        continue
    
    #-------------------temp code start
    img_data = cv2.imread(BKG_IMG_DIR + bkg_images[8])
    #-------------------temp code end
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    
    this_frame_bboxes = []
    boxes = []
    objs = []
    b = []
    #wrong_detection_objs = [[130, 311, 226, 475]]
    #objs_iter = root.iter('object')
    N = 0
    array_object = root.findall('object')
    for obj in array_object:
        erro_obj_removed = False
        stable_bbox_removed = False
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        this_frame_bboxes.append(b)
        if(len(last_frame_bboxes)>0):
            for stable_bbox in last_frame_bboxes:
                if(IOU(stable_bbox, b) > 0.85):
                    #print(b)
                    root.remove(obj)
                    stable_bbox_removed = True
                    N = N+1
                    break
            if(stable_bbox_removed):
                continue
        #filter size which smaller than 16X16 fit for SSD300
        bbox_width = b[1] - b[0]
        bbox_heigt = b[3] - b[2]
        if(not(bbox_width > 16 and bbox_heigt > 16)):
            print('remove:{0}:{1}'.format(bbox_width, bbox_heigt))
            root.remove(obj)
            continue
        else:
            for box in boxes:
                if(IOU(box, b) > 0.6):
                    print('remove:{0}:{1}'.format(bbox_width, bbox_heigt))
                    root.remove(obj)
                    continue
            print('add:{0}:{1}'.format(bbox_width, bbox_heigt))          
        boxes.append(b)
        objs += obj
    last_frame_bboxes = this_frame_bboxes.copy()  
    #print(boxes)
    
    if(len(boxes) == 0):
        continue
    past_to_background_from_image_file(SEED_IMG_DIR + xml_name[:-3] + 'jpg', boxes, img_data)
    tree.write(GENE_XML_DIR + xml_name)

add:47.0:44.0
add:26.0:24.0
add:59.0:44.0
add:28.0:22.0
add:52.0:51.0
add:31.0:20.0
add:40.0:22.0
add:31.0:24.0
add:29.0:18.0
add:32.0:20.0
add:131.0:131.0
add:281.0:181.0
add:84.0:75.0
add:54.0:41.0
add:45.0:44.0
add:42.0:38.0
add:25.0:47.0
add:56.0:53.0
add:57.0:49.0
add:38.0:26.0
add:26.0:24.0
add:41.0:35.0
add:36.0:19.0
add:42.0:31.0
add:45.0:29.0
add:39.0:34.0
add:42.0:30.0
add:40.0:27.0
add:69.0:53.0
add:56.0:59.0
add:87.0:65.0
add:46.0:92.0
add:138.0:135.0
add:81.0:60.0
add:55.0:39.0
add:46.0:42.0
add:44.0:44.0
add:61.0:60.0
add:89.0:83.0
add:30.0:19.0
add:71.0:58.0
add:30.0:18.0
add:60.0:36.0
add:42.0:29.0
add:40.0:25.0
add:78.0:71.0
add:39.0:55.0
add:36.0:23.0
add:54.0:51.0
add:39.0:48.0
add:45.0:28.0
add:41.0:41.0
add:55.0:52.0
add:142.0:136.0
add:41.0:26.0
add:76.0:48.0
add:41.0:26.0
add:38.0:25.0
add:64.0:52.0
add:75.0:35.0
add:49.0:39.0
add:36.0:30.0
add:40.0:31.0
add:38.0:35.0
add:38.0:26.0
add:34.0:22.0
add:25.0:17.0
add:32.0:24.0
add:45.0:33.0
add:61.0:42.0
add:81.0:53.

add:36.0:19.0
add:52.0:59.0
add:30.0:18.0
add:49.0:23.0
add:41.0:30.0
remove:23.0:11.0
add:31.0:20.0
add:34.0:23.0
add:26.0:17.0
add:27.0:18.0
remove:23.0:12.0
add:30.0:20.0
remove:13.0:9.0
add:40.0:23.0
remove:15.0:11.0
remove:23.0:13.0
add:54.0:62.0
add:41.0:28.0
add:28.0:20.0
add:143.0:155.0
add:54.0:37.0
remove:27.0:15.0
remove:18.0:11.0
add:26.0:17.0
remove:25.0:11.0
remove:23.0:14.0
add:28.0:20.0
add:58.0:36.0
remove:25.0:15.0
remove:28.0:16.0
add:54.0:50.0
add:32.0:18.0
add:28.0:18.0
add:28.0:20.0
remove:25.0:16.0
add:36.0:24.0
add:33.0:18.0
add:75.0:41.0
add:28.0:19.0
add:52.0:60.0
add:30.0:19.0
remove:26.0:15.0
add:38.0:30.0
add:31.0:21.0
remove:22.0:13.0
add:63.0:35.0
remove:27.0:15.0
add:30.0:19.0
add:54.0:54.0
add:48.0:47.0
add:106.0:91.0
add:29.0:96.0
add:29.0:20.0
add:28.0:20.0
add:26.0:17.0
add:55.0:55.0
add:40.0:26.0
add:28.0:21.0
remove:19.0:13.0
add:73.0:39.0
add:32.0:21.0
add:33.0:19.0
add:31.0:19.0
remove:28.0:14.0
add:55.0:40.0
remove:23.0:13.0
remove:24.0:13.0
add

add:151.0:160.0
add:55.0:52.0
add:39.0:30.0
add:41.0:18.0
add:40.0:35.0
add:40.0:33.0
add:36.0:23.0
add:124.0:89.0
add:62.0:52.0
add:53.0:38.0
add:46.0:25.0
add:70.0:65.0
add:66.0:39.0
add:28.0:24.0
add:35.0:26.0
add:122.0:66.0
add:78.0:64.0
add:44.0:34.0
add:37.0:28.0
add:39.0:30.0
add:32.0:24.0
remove:22.0:13.0
add:59.0:64.0
add:34.0:32.0
remove:17.0:11.0
remove:17.0:11.0
add:61.0:61.0
add:43.0:32.0
add:35.0:21.0
remove:29.0:16.0
add:27.0:19.0
add:31.0:19.0
add:26.0:20.0
add:21.0:17.0
add:28.0:17.0
add:35.0:22.0
add:56.0:116.0
add:51.0:36.0
remove:23.0:12.0
add:134.0:86.0
add:70.0:66.0
add:29.0:22.0
add:55.0:39.0
add:25.0:20.0
add:41.0:23.0
add:53.0:57.0
add:21.0:25.0
add:34.0:21.0
add:120.0:95.0
add:263.0:173.0
add:29.0:25.0
add:45.0:29.0
add:40.0:29.0
add:86.0:89.0
add:98.0:101.0
add:57.0:44.0
add:69.0:50.0
add:36.0:35.0
add:61.0:45.0
add:45.0:31.0
add:26.0:19.0
add:262.0:310.0
add:146.0:107.0
add:37.0:22.0
add:37.0:24.0
add:157.0:226.0
add:48.0:23.0
add:25.0:20.0
add:62.0:50.0
add

add:31.0:23.0
add:45.0:25.0
add:46.0:39.0
add:44.0:32.0
add:30.0:17.0
add:73.0:62.0
add:35.0:32.0
add:36.0:29.0
add:55.0:33.0
add:50.0:46.0
add:32.0:20.0
add:40.0:23.0
add:33.0:26.0
add:80.0:30.0
add:34.0:37.0
remove:26.0:15.0
add:70.0:41.0
add:95.0:90.0
remove:95.0:90.0
add:95.0:90.0
add:32.0:19.0
add:29.0:22.0
remove:27.0:16.0
add:39.0:24.0
add:44.0:31.0
add:31.0:22.0
remove:28.0:16.0
add:34.0:17.0
add:26.0:19.0
remove:15.0:9.0
add:82.0:57.0
add:44.0:21.0
add:33.0:37.0
add:44.0:39.0
add:36.0:23.0
add:36.0:20.0
add:30.0:23.0
remove:25.0:12.0
add:39.0:25.0
add:33.0:35.0
add:37.0:20.0
add:42.0:26.0
add:71.0:38.0
remove:27.0:15.0
add:27.0:19.0
add:28.0:17.0
add:27.0:19.0
add:29.0:22.0
add:42.0:26.0
add:29.0:26.0
add:28.0:19.0
add:37.0:22.0
add:48.0:23.0
add:33.0:34.0
remove:17.0:12.0
add:27.0:20.0
add:28.0:17.0
add:29.0:20.0
add:31.0:21.0
add:27.0:17.0
add:32.0:34.0
add:24.0:19.0
remove:17.0:15.0
add:33.0:36.0
add:65.0:47.0
add:37.0:21.0
add:29.0:21.0
add:19.0:26.0
add:37.0:25.0
remove:1

add:60.0:63.0
add:31.0:30.0
add:242.0:264.0
add:30.0:29.0
add:195.0:289.0
add:24.0:19.0
add:38.0:30.0
add:129.0:159.0
add:35.0:24.0
add:26.0:18.0
add:29.0:22.0
add:26.0:21.0
add:34.0:26.0
add:85.0:103.0
add:31.0:22.0
add:22.0:17.0
add:26.0:20.0
add:123.0:144.0
add:73.0:68.0
add:27.0:22.0
add:91.0:99.0
add:27.0:22.0
add:60.0:60.0
add:20.0:21.0
add:32.0:20.0
add:25.0:17.0
add:28.0:20.0
add:73.0:46.0
remove:17.0:11.0
add:133.0:135.0
add:40.0:38.0
add:36.0:23.0
add:79.0:70.0
add:20.0:21.0
add:108.0:133.0
add:136.0:121.0
add:31.0:23.0
add:41.0:24.0
add:28.0:19.0
add:43.0:40.0
add:40.0:30.0
add:27.0:18.0
add:44.0:28.0
remove:25.0:16.0
remove:16.0:9.0
remove:19.0:10.0
add:36.0:26.0
add:33.0:20.0
add:26.0:21.0
add:25.0:22.0
add:38.0:31.0
add:82.0:86.0
add:112.0:117.0
remove:13.0:12.0
add:27.0:18.0
add:57.0:55.0
add:28.0:20.0
add:30.0:19.0
add:25.0:19.0
add:29.0:19.0
add:21.0:25.0
remove:27.0:16.0
add:17.0:18.0
add:25.0:20.0
add:74.0:55.0
add:33.0:24.0
add:24.0:18.0
remove:21.0:14.0
add:142.0:7

...

add:27.0:17.0
add:20.0:32.0
add:30.0:20.0
add:20.0:19.0
add:28.0:17.0
add:27.0:19.0
remove:21.0:11.0
add:160.0:100.0
add:30.0:19.0
add:40.0:25.0
add:25.0:17.0
add:32.0:19.0
add:33.0:21.0
add:37.0:23.0
add:32.0:22.0
add:19.0:18.0
remove:30.0:15.0
add:42.0:23.0
add:42.0:29.0
remove:30.0:15.0
add:46.0:24.0
add:18.0:18.0
remove:19.0:10.0
add:34.0:24.0
add:44.0:23.0
add:63.0:49.0
add:27.0:17.0
add:78.0:54.0
remove:43.0:16.0
add:54.0:42.0
remove:25.0:13.0
add:26.0:20.0
add:19.0:19.0
remove:26.0:15.0
add:67.0:55.0
add:45.0:71.0
add:19.0:18.0
add:33.0:18.0
add:73.0:57.0
add:17.0:50.0
add:19.0:18.0
remove:20.0:11.0
add:42.0:22.0
add:32.0:23.0
remove:19.0:14.0
add:21.0:18.0
add:75.0:55.0
add:25.0:20.0
remove:24.0:12.0
remove:17.0:11.0
add:29.0:18.0
add:61.0:44.0
add:78.0:62.0
remove:21.0:10.0
add:20.0:18.0
remove:22.0:11.0
add:34.0:23.0
add:40.0:20.0
remove:31.0:15.0
remove:35.0:16.0
add:37.0:27.0
remove:32.0:14.0
add:25.0:18.0
add:48.0:28.0
remove:31.0:16.0
add:27.0:60.0
add:53.0:44.0
add:47.0:

add:38.0:24.0
remove:26.0:15.0
add:28.0:17.0
add:58.0:46.0
add:31.0:17.0
add:46.0:24.0
add:36.0:23.0
remove:29.0:16.0
add:45.0:36.0
add:40.0:28.0
add:21.0:18.0
add:49.0:30.0
add:42.0:19.0
add:43.0:29.0
add:20.0:18.0
add:54.0:29.0
remove:29.0:16.0
add:37.0:20.0
remove:16.0:17.0
add:18.0:22.0
add:39.0:23.0
remove:27.0:15.0
add:36.0:33.0
add:32.0:21.0
add:45.0:24.0
add:36.0:23.0
add:40.0:25.0
remove:26.0:15.0
remove:16.0:18.0
remove:18.0:9.0
remove:16.0:12.0
add:54.0:33.0
add:33.0:18.0
add:21.0:22.0
add:32.0:21.0
add:27.0:17.0
add:59.0:60.0
add:29.0:27.0
remove:23.0:11.0
add:61.0:44.0
remove:28.0:15.0
add:70.0:32.0
remove:17.0:12.0
add:59.0:47.0
add:31.0:17.0
remove:20.0:14.0
add:32.0:19.0
add:31.0:22.0
remove:19.0:14.0
add:42.0:54.0
remove:32.0:16.0
remove:15.0:10.0
remove:16.0:9.0
add:28.0:17.0
add:44.0:19.0
add:31.0:21.0
add:29.0:19.0
add:64.0:31.0
add:42.0:26.0
add:38.0:23.0
add:87.0:95.0
remove:17.0:12.0
add:215.0:177.0
add:28.0:18.0
add:63.0:48.0
add:29.0:18.0
add:64.0:38.0
add:218.

add:64.0:65.0
remove:14.0:27.0
add:52.0:65.0
remove:34.0:15.0
add:41.0:48.0
add:30.0:21.0
add:33.0:27.0
remove:21.0:12.0
add:38.0:30.0
remove:23.0:14.0
add:27.0:21.0
add:165.0:94.0
add:33.0:21.0
remove:21.0:10.0
add:31.0:23.0
add:28.0:19.0
add:28.0:21.0
add:34.0:23.0
remove:18.0:9.0
add:135.0:147.0
add:27.0:19.0
add:32.0:28.0
remove:24.0:13.0
add:39.0:29.0
add:22.0:22.0
add:129.0:79.0
add:34.0:26.0
add:28.0:21.0
add:28.0:23.0
add:115.0:131.0
add:42.0:26.0
remove:15.0:9.0
remove:18.0:13.0
remove:16.0:10.0
add:27.0:19.0
add:34.0:22.0
add:52.0:40.0
add:34.0:21.0
remove:21.0:13.0
remove:20.0:12.0
add:29.0:20.0
add:74.0:67.0
add:36.0:30.0
add:28.0:19.0
remove:25.0:14.0
add:36.0:27.0
add:26.0:19.0
add:29.0:22.0
add:31.0:29.0
add:20.0:20.0
add:27.0:19.0
add:33.0:22.0
add:26.0:18.0
add:27.0:20.0
remove:25.0:16.0
remove:17.0:11.0
remove:22.0:13.0
add:22.0:17.0
remove:17.0:11.0
add:51.0:48.0
add:54.0:43.0
remove:16.0:10.0
add:27.0:19.0
add:41.0:26.0
remove:18.0:10.0
add:48.0:32.0
remove:22.0:11.

add:29.0:18.0
remove:17.0:12.0
remove:22.0:14.0
remove:20.0:11.0
remove:15.0:9.0
add:25.0:20.0
add:33.0:21.0
remove:14.0:9.0
remove:13.0:10.0
remove:15.0:10.0
remove:11.0:9.0
add:43.0:27.0
add:20.0:22.0
remove:17.0:11.0
remove:18.0:11.0
remove:15.0:9.0
remove:16.0:13.0
add:44.0:31.0
add:55.0:37.0
add:45.0:33.0
add:68.0:48.0
remove:28.0:14.0
add:25.0:27.0
remove:14.0:12.0
remove:13.0:10.0
add:29.0:17.0
add:32.0:21.0
add:45.0:21.0
add:44.0:24.0
remove:22.0:10.0
add:44.0:68.0
add:35.0:23.0
add:32.0:20.0
remove:25.0:16.0
remove:29.0:16.0
remove:16.0:11.0
remove:13.0:9.0
remove:15.0:10.0
remove:20.0:12.0
add:43.0:23.0
add:33.0:22.0
remove:19.0:11.0
remove:22.0:12.0
remove:21.0:15.0
add:29.0:22.0
remove:25.0:12.0
add:28.0:17.0
remove:19.0:12.0
remove:22.0:12.0
add:63.0:52.0
add:27.0:18.0
add:33.0:22.0
remove:18.0:12.0
remove:20.0:12.0
remove:20.0:12.0
add:44.0:31.0
add:26.0:17.0
add:32.0:22.0
remove:15.0:11.0
add:38.0:25.0
remove:22.0:11.0
add:31.0:23.0
add:34.0:21.0
remove:19.0:12.0
remove:

add:41.0:29.0
add:30.0:22.0
add:25.0:18.0
add:26.0:18.0
add:215.0:311.0
add:123.0:197.0
add:28.0:26.0
add:23.0:21.0
add:79.0:127.0
add:26.0:21.0
add:136.0:117.0
add:94.0:92.0
add:57.0:87.0
remove:16.0:14.0
add:47.0:61.0
add:43.0:54.0
remove:14.0:9.0
remove:43.0:54.0
add:43.0:54.0
add:31.0:21.0
add:32.0:24.0
add:27.0:22.0
add:32.0:20.0
add:24.0:20.0
add:27.0:17.0
remove:11.0:9.0
add:31.0:22.0
add:55.0:66.0
add:122.0:70.0
add:35.0:32.0
remove:22.0:12.0
add:33.0:25.0
add:29.0:28.0
add:208.0:87.0
remove:17.0:12.0
add:26.0:20.0
add:21.0:21.0
add:35.0:23.0
remove:19.0:11.0
add:30.0:20.0
add:158.0:85.0
add:20.0:17.0
add:30.0:17.0
add:111.0:70.0
remove:21.0:16.0
add:33.0:21.0
remove:19.0:12.0
add:35.0:24.0
add:50.0:45.0
add:32.0:22.0
remove:20.0:10.0
remove:21.0:12.0
remove:17.0:9.0
remove:16.0:10.0
add:38.0:27.0
remove:19.0:12.0
add:48.0:47.0
add:37.0:23.0
remove:19.0:11.0
add:29.0:17.0
remove:17.0:10.0
remove:18.0:12.0
remove:27.0:15.0
add:25.0:20.0
remove:17.0:12.0
add:25.0:18.0
add:28.0:19